<a href="https://colab.research.google.com/github/Janina712/MLTSA_FinalProject/blob/main/edf_tse_opening.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Here we open all downloaded `.edf` and `.tse` files and put them in a dataframe 

In [52]:
pip install mne

     |████████████████████████████████| 7.5 MB 16.3 MB/s 


In [72]:
import requests
from bs4 import BeautifulSoup
from pathlib import Path
from tqdm import tqdm
import shutil
from collections import Counter
import pandas as pd
import mne
import numpy as np

tqdm.pandas()

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [58]:
drive_tuh_eeg_loc = Path('/content/drive/MyDrive/Classes/22S-PHYS-667/project/isip_tuh_eeg')

In [57]:
working_folder = Path('/content/drive/MyDrive/Classes/22S-PHYS-667/project/temp_data')

## Reading files from the folder

**Input**: a folder with files

**Output**: dataframe with parsed filenames for further aggregation

In [5]:
downloaded_files = list(Path(drive_tuh_eeg_loc).glob('*'))
downloaded_files[:3]

[PosixPath('/content/drive/MyDrive/Classes/22S-PHYS-667/project/isip_tuh_eeg/train__02_tcp_le__058__00005804__s002_2008_11_05__00005804_s002_t003.tse'),
 PosixPath('/content/drive/MyDrive/Classes/22S-PHYS-667/project/isip_tuh_eeg/train__02_tcp_le__058__00005804__s002_2008_11_05__00005804_s002_t004.tse'),
 PosixPath('/content/drive/MyDrive/Classes/22S-PHYS-667/project/isip_tuh_eeg/train__02_tcp_le__058__00005804__s002_2008_11_05__00005804_s002_t005.tse')]

In [6]:
# how many files we have in total
len(downloaded_files)

13270

In [17]:
files_raw_df = pd.DataFrame(
  [fpath.name for fpath in downloaded_files], 
  columns = ['filename']
)

In [18]:
len(downloaded_files)

13270

In [23]:
files_raw_df

,filename
0,train__02_tcp_le__058__00005804__s002_2008_11_...
1,train__02_tcp_le__058__00005804__s002_2008_11_...
2,train__02_tcp_le__058__00005804__s002_2008_11_...
3,train__02_tcp_le__058__00005804__s002_2008_11_...
4,train__02_tcp_le__060__00006083__s003_2010_10_...
...,...
13265,dev__01_tcp_ar__085__00008512__s002_2012_01_09...
13266,dev__01_tcp_ar__085__00008512__s002_2012_01_09...
13267,dev__01_tcp_ar__085__00008512__s002_2012_01_09...
13268,dev__01_tcp_ar__085__00008512__s002_2012_01_09...


In [20]:
files_raw_df['filename'][4].split('__')

['train',
 '02_tcp_le',
 '060',
 '00006083',
 's003_2010_10_11',
 '00006083_s003_t001.tse']

In [33]:
def split_fname(row):

  split_row_str = row['filename'].split('__')

  fname, file_ext = split_row_str[-1].split('.')

  row['category'] = split_row_str[0]
  row['1_idk'] = split_row_str[1]
  row['2_idk'] = split_row_str[2]
  row['3_idk'] = split_row_str[3]
  row['4_idk'] = split_row_str[4]
  row['file_name'] = fname #fname WITHOUT file extension
  row['file_ext'] = file_ext #file extension (.edf or .tse)

  return row

In [35]:
# not the most efficient way, but it's expressive

files_raw_df = files_raw_df.progress_apply(
  split_fname, 
  axis=1
)

100%|██████████| 13270/13270 [00:53<00:00, 245.82it/s]


In [39]:
files_raw_df

,filename,category,1_idk,2_idk,3_idk,4_idk,file_name,file_ext
0,train__02_tcp_le__058__00005804__s002_2008_11_...,train,02_tcp_le,058,00005804,s002_2008_11_05,00005804_s002_t003,tse
1,train__02_tcp_le__058__00005804__s002_2008_11_...,train,02_tcp_le,058,00005804,s002_2008_11_05,00005804_s002_t004,tse
2,train__02_tcp_le__058__00005804__s002_2008_11_...,train,02_tcp_le,058,00005804,s002_2008_11_05,00005804_s002_t005,tse
3,train__02_tcp_le__058__00005804__s002_2008_11_...,train,02_tcp_le,058,00005804,s002_2008_11_05,00005804_s002_t006,tse
4,train__02_tcp_le__060__00006083__s003_2010_10_...,train,02_tcp_le,060,00006083,s003_2010_10_11,00006083_s003_t001,tse
...,...,...,...,...,...,...,...,...
13265,dev__01_tcp_ar__085__00008512__s002_2012_01_09...,dev,01_tcp_ar,085,00008512,s002_2012_01_09,00008512_s002_t000,edf
13266,dev__01_tcp_ar__085__00008512__s002_2012_01_09...,dev,01_tcp_ar,085,00008512,s002_2012_01_09,00008512_s002_t001,edf
13267,dev__01_tcp_ar__085__00008512__s002_2012_01_09...,dev,01_tcp_ar,085,00008512,s002_2012_01_09,00008512_s002_t002,edf
13268,dev__01_tcp_ar__085__00008512__s002_2012_01_09...,dev,01_tcp_ar,085,00008512,s002_2012_01_09,00008512_s002_t003,edf


### Saving and loading dataframe

In [45]:
# saving result

files_raw_df.to_pickle(
    Path(working_folder).joinpath('files_raw_df.pkl')
)

In [46]:
# saving result

files_raw_df = pd.read_pickle(
    Path(working_folder).joinpath('files_raw_df.pkl')
)

In [47]:
files_raw_df

,filename,category,1_idk,2_idk,3_idk,4_idk,file_name,file_ext
0,train__02_tcp_le__058__00005804__s002_2008_11_...,train,02_tcp_le,058,00005804,s002_2008_11_05,00005804_s002_t003,tse
1,train__02_tcp_le__058__00005804__s002_2008_11_...,train,02_tcp_le,058,00005804,s002_2008_11_05,00005804_s002_t004,tse
2,train__02_tcp_le__058__00005804__s002_2008_11_...,train,02_tcp_le,058,00005804,s002_2008_11_05,00005804_s002_t005,tse
3,train__02_tcp_le__058__00005804__s002_2008_11_...,train,02_tcp_le,058,00005804,s002_2008_11_05,00005804_s002_t006,tse
4,train__02_tcp_le__060__00006083__s003_2010_10_...,train,02_tcp_le,060,00006083,s003_2010_10_11,00006083_s003_t001,tse
...,...,...,...,...,...,...,...,...
13265,dev__01_tcp_ar__085__00008512__s002_2012_01_09...,dev,01_tcp_ar,085,00008512,s002_2012_01_09,00008512_s002_t000,edf
13266,dev__01_tcp_ar__085__00008512__s002_2012_01_09...,dev,01_tcp_ar,085,00008512,s002_2012_01_09,00008512_s002_t001,edf
13267,dev__01_tcp_ar__085__00008512__s002_2012_01_09...,dev,01_tcp_ar,085,00008512,s002_2012_01_09,00008512_s002_t002,edf
13268,dev__01_tcp_ar__085__00008512__s002_2012_01_09...,dev,01_tcp_ar,085,00008512,s002_2012_01_09,00008512_s002_t003,edf


### Putting all `.edf` objects in a dict

In [55]:
files_raw_df['filename']

0        train__02_tcp_le__058__00005804__s002_2008_11_...
1        train__02_tcp_le__058__00005804__s002_2008_11_...
2        train__02_tcp_le__058__00005804__s002_2008_11_...
3        train__02_tcp_le__058__00005804__s002_2008_11_...
4        train__02_tcp_le__060__00006083__s003_2010_10_...
                               ...                        
13265    dev__01_tcp_ar__085__00008512__s002_2012_01_09...
13266    dev__01_tcp_ar__085__00008512__s002_2012_01_09...
13267    dev__01_tcp_ar__085__00008512__s002_2012_01_09...
13268    dev__01_tcp_ar__085__00008512__s002_2012_01_09...
13269    dev__01_tcp_ar__085__00008512__s002_2012_01_09...
Name: filename, Length: 13270, dtype: object

In [106]:
file_path = drive_tuh_eeg_loc.joinpath(files_raw_df['filename'][7332])
file_path

PosixPath('/content/drive/MyDrive/Classes/22S-PHYS-667/project/isip_tuh_eeg/train__02_tcp_le__053__00005347__s001_2008_11_24__00005347_s001_t001.edf')

In [120]:
edf_fnames_list = files_raw_df[
  files_raw_df['file_ext'] == 'edf'
]['filename']

raw_files_dict = dict()
for fname in tqdm(edf_fnames_list[:50]):
  data_obj = mne.io.read_raw_edf(
    file_path,
    verbose=False,
  )
  raw_files_dict[fname] = data_obj

100%|██████████| 50/50 [00:00<00:00, 106.44it/s]


In [123]:
## access by filename

raw_files_dict[
  'train__03_tcp_ar_a__072__00007221__s003_2010_11_01__00007221_s003_t004.edf'
]

<RawEDF | train__02_tcp_le__053__00005347__s001_2008_11_24__00005347_s001_t001.edf, 41 x 234000 (936.0 s), ~44 kB, data not loaded>